# VII. Types, type hierarchy, and multiple dispatch

In Julia types play a central role. We've covered enough so far that you can write Julia code as you would in other languages, but having an understanding of types will give you a better understanding of how Julia works and hopefully help you write better Julia code. 

In Julia, everything has a type. As we've already seen, you can figure out an object's type using the __typeof__ funciton.

In [1]:
typeof( 1.3 )

Float64

In [2]:
typeof( 22 )

Int64

In [3]:
typeof( "julia" )

String

In [4]:
typeof( 3//2 )

Rational{Int64}

In [5]:
typeof( 3 + 2im )

Complex{Int64}

In [6]:
typeof( randn( 5, 5 ) )

Array{Float64,2}

The curly braces indicate the type is a **parametric** type. So *Array* is a parametric type since it's a type with type parameters (in this case *Float64* and *2*).

You can use the __eltype__ function to determine the element type.

In [7]:
A = rand( 5, 5 )

5×5 Array{Float64,2}:
 0.340621  0.560391  0.723921  0.517655   0.518386 
 0.49682   0.74783   0.373878  0.94408    0.515213 
 0.542565  0.226895  0.584024  0.0248887  0.0199738
 0.308699  0.88129   0.781702  0.416065   0.0999678
 0.427554  0.74992   0.436451  0.538462   0.17086  

In [8]:
eltype( A )

Float64

In [9]:
t = ( "summer", 32, 4.3 )

("summer", 32, 4.3)

In [10]:
typeof( t )

Tuple{String,Int64,Float64}

In [11]:
eltype( t )

Any

Because of Julia's emphasis on types you can sometimes run into unexpected problems. Let's generate a random array of integers between 1 and 20:

In [12]:
A = rand( 1 : 20, 5, 5 )

5×5 Array{Int64,2}:
 11   7   9  14  19
  1  18  12  20  20
 14  15  20  12  19
  5  16   9   4  20
  2   9   6  18  16

As stated before, arrays are mutable:

In [13]:
A[ 1, 1 ] = 19

19

In [14]:
A

5×5 Array{Int64,2}:
 19   7   9  14  19
  1  18  12  20  20
 14  15  20  12  19
  5  16   9   4  20
  2   9   6  18  16

Let's now set element A[1,2] equal to 3.2:

In [15]:
A[ 1, 2 ] = 3.2

InexactError: InexactError: Int64(3.2)

What happened here?

### How types are organized in Julia:

In Julia, types are organized according according to a tree like  structure. At the top of the tree is the *Any* type. So all types are subtypes of *Any*. At the bottom of the tree (the leaves of the tree) are __concrete types__, i.e. Float64, Int64, UInt32, etc. A type can have at most one parent, but possibly more than one child.

To get a sense of how the type hierarchy is organized you can use the functions __subtypes__ and __supertype__. Let's start with a simple concrete type *Float64* and traverse the tree upwards.

In [16]:
supertype( Float64 )

AbstractFloat

So the parent of *Float64* is a type called *AbstractFloat* which in Julia is called an **abstract type**.

In [17]:
supertype( AbstractFloat )

Real

In [18]:
supertype( Real )

Number

In [19]:
supertype( Number )

Any

In [20]:
supertype( Any )

Any

From the above ouptut, we see that *Real*, *Number*, etc. are also **abstract types** (i.e. they can have children). When you create variables in Julia they can only be __concrete types__, i.e. you can not instantiate a variable of type *Real*.

Let's now start with the *Real* type and use the **subtypes** function to go down the tree.

In [21]:
subtypes( Real )

4-element Array{Any,1}:
 AbstractFloat     
 AbstractIrrational
 Integer           
 Rational          

So *Real* has four subtypes: *AbstractFloat*, *Integer*, *Irrational*, *Rational*.

In [22]:
subtypes( Integer )

3-element Array{Any,1}:
 Bool    
 Signed  
 Unsigned

In [23]:
subtypes( Signed )

6-element Array{Any,1}:
 BigInt
 Int128
 Int16 
 Int32 
 Int64 
 Int8  

In [24]:
subtypes( Int64 )

0-element Array{Type,1}

We can see that *Int64* is a **concrete type**. There are also suptype and supertype operators you can use to ask questions if a type is a supertype or subtype of some other type. Here we test if *Real* is a subtype of *Number*:

In [25]:
Real <: Number

true

Is *Int64* a subtype of *Float64*?

In [26]:
Int64 <: Float64

false

Let's test if *Real* is a supertype of *Rational*:

In [27]:
Real >: Rational

true

Is *Int64* a supertype of *Complex*?

In [28]:
Int64 >: Complex

false

Types are important in Julia for many reasons, but perhaps most obviously due to Julia's approach to calling functions known as **multiple dispatch**.

In Julia many versions of the same named function can exist and these different versions are called **methods**. The methods differ in that they have different function signatures. In general, these different methods correspond to a specific type of behavior by the function determined by the type of inputs.

You can see this with the built-in multiplication function __*__:

In [29]:
*( 3, 2.2 )

6.6000000000000005

In [30]:
*( "string","input" )

"stringinput"

As we can see the behavior of __*__ is different depending on the input types: for numbers it does multiplication and for strings it does concatenation. You can see all the methods for a function using **methods**:

In [31]:
methods( * )

# 358 methods for generic function "*":
[1] *(x::Bool, z::Complex{Bool}) in Base at complex.jl:286
[2] *(x::Bool, y::Bool) in Base at bool.jl:98
[3] *(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:110
[4] *(x::Bool, z::Complex) in Base at complex.jl:293
[5] *(x::Bool, y::AbstractIrrational) in Base at irrationals.jl:139
[6] *(a::Float16, b::Float16) in Base at float.jl:398
[7] *(x::Float32, y::Float32) in Base at float.jl:404
[8] *(x::Float64, y::Float64) in Base at float.jl:405
[9] *(z::Complex{Bool}, x::Bool) in Base at complex.jl:287
[10] *(z::Complex{Bool}, x::Real) in Base at complex.jl:305
[11] *(r::Regex) in Base at regex.jl:660
[12] *(::Missing, ::Missing) in Base at missing.jl:114
[13] *(::Missing, ::Number) in Base at missing.jl:115
[14] *(d::Missing, x::AbstractString) in Base at missing.jl:170
[15] *(this::Base.Grisu.Float, other::Base.Grisu.Float) in Base.Grisu at grisu/float.jl:140
[16] *(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:413
[17] *(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:503
[18] *(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:502
[19] *(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:501
[20] *(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:472
[21] *(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:520
[22] *(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:522
[23] *(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:563
[24] *(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:556
[25] *(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:550
[26] *(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:409
[27] *(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:378
[28] *(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:385
[29] *(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:393
[30] *(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:401
[31] *(B::BitArray{2}, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/uniformscaling.jl:195
[32] *(y::AbstractFloat, x::Bool) in Base at bool.jl:112
[33] *(x::T, y::T) where T<:Union{Int128, UInt128} in Base at int.jl:859
[34] *(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:54
[35] *(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:521
[36] *(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:523
[37] *(a::Integer, b::Integer) in Base at int.jl:871
[38] *(x::Real, r::StepRangeLen{#s66,#s65,S} where S where #s65<:Base.TwicePrecision where #s66<:Real) in Base at twiceprecision.jl:479
[39] *(z::Complex, w::Complex) in Base at complex.jl:273
[40] *(z::Complex, x::Bool) in Base at complex.jl:294
[41] *(x::Real, z::Complex{Bool}) in Base at complex.jl:304
[42] *(x::Real, z::Complex) in Base at complex.jl:316
[43] *(z::Complex, x::Real) in Base at complex.jl:317
[44] *(x::Rational, y::Rational) in Base at rational.jl:264
[45] *(a::Integer, index::CartesianIndex{N}) where N in Base.IteratorsMD at multidimensional.jl:119
[46] *(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:389
[47] *(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:397
[48] *(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:405
[49] *(x::AbstractIrrational, y::AbstractIrrational) in Base at irrationals.jl:137
[50] *(y::Real, x::Dates.Period) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/periods.jl:93
[51] *(x::Real, A::LinearAlgebra.Hermitian) in LinearAlgebra at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/LinearAlgebra/src/symmetric.jl:494
[52] *(x::Number) in Base at operators.

What Julia does at runtime is call the specialized version of the function (i.e. method) that corresponds to the type of input arguments being passed.

You can incorporate this type of behavior into your own functions. Let's write a function that takes a Float64 squares it, and returns the result.

In [32]:
function MySquareSpecific( x::Float64 )
    return x^2
end

MySquareSpecific (generic function with 1 method)

Passing a *Float64* should work, but passing an *Int64* will not:

In [33]:
MySquareSpecific( 3.2 )

10.240000000000002

In [34]:
MySquareSpecific( 2 )

MethodError: MethodError: no method matching MySquareSpecific(::Int64)
Closest candidates are:
  MySquareSpecific(!Matched::Float64) at In[32]:2

To remedy this we could write another method for MySquuareSpecific to work on an Int64.

In [35]:
function MySquareSpecific( x::Int64 )
    return x^2
end

MySquareSpecific (generic function with 2 methods)

Our function now has two methods:

In [36]:
methods( MySquareSpecific )

# 2 methods for generic function "MySquareSpecific":
[1] MySquareSpecific(x::Int64) in Main at In[35]:2
[2] MySquareSpecific(x::Float64) in Main at In[32]:2

In [37]:
round(MySquareSpecific( 3.2 ), digits = 4 )

10.24

In [38]:
MySquareSpecific( 2 )

4

Though the function will not work on arguments of the Rational type which is something we might want.

In [39]:
MySquareSpecific( 3//2 )

MethodError: MethodError: no method matching MySquareSpecific(::Rational{Int64})
Closest candidates are:
  MySquareSpecific(!Matched::Int64) at In[35]:2
  MySquareSpecific(!Matched::Float64) at In[32]:2

Instead of writing multiple versions of our function we could've written a generalized version to work on anything that's a subtype of *Real*.

In [40]:
function MySquareGen( x :: Real )
    return x .^ 2
end

MySquareGen (generic function with 1 method)

In [41]:
a, b, c = MySquareGen( 3.2 ), MySquareGen( 2 ), MySquareGen( 3//2 )

(10.240000000000002, 4, 9//4)

However, in our case the best thing to do would be to leave the input type unspecified and simply do:

In [42]:
function MySquare(x)
    return x^2
end

MySquare (generic function with 1 method)

Reason being, Julia will do type inference on the argument being passed in. The first time it encounters this argument type it will infer the argument type, then compile and cache this compiled version of the function for the inferred argument type. The next time the function is called with the same argument type Julia will call the specialized compiled version that it has stored in memory.

For your reference, if you wanted to define a function whose input was an Array of Float64s that would look something like the following:

In [43]:
function ArrFloat( A :: Array{ Float64 } )
    #statements
end

ArrFloat (generic function with 1 method)

And if you wanted to define a function whose input was an Array whose elements are restricted to being subtypes of *Real*:

In [44]:
function ArrReal( A :: Array{ Real } )
     #statements
end

ArrReal (generic function with 1 method)

# Exercise 6
* Find the supertype of the Bool type.
* Is Int64 subtype of Bool?
* Display all the methods for the "isless" function.

In this lesson we covered:
* Julia types
* Type operations
* Mulitple dispatch